In [22]:
import pandas as pd
fake_real_df = pd.read_csv('./fake_or_real_news.csv')

In [37]:
fake_real_df

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL
5,6903,"Tehran, USA","\nI’m not an immigrant, but my grandparents ...",FAKE
6,7341,Girl Horrified At What She Watches Boyfriend D...,"Share This Baylee Luciani (left), Screenshot o...",FAKE
7,95,‘Britain’s Schindler’ Dies at 106,A Czech stockbroker who saved more than 650 Je...,REAL
8,4869,Fact check: Trump and Clinton at the 'commande...,Hillary Clinton and Donald Trump made some ina...,REAL
9,2909,Iran reportedly makes new push for uranium con...,Iranian negotiators reportedly have made a las...,REAL


In [23]:
fake_real_df_clean = fake_real_df[['title','text','label']]

In [24]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(fake_real_df_clean, fake_real_df_clean["label"]):
    strat_train_set = fake_real_df_clean.loc[train_index]
    strat_test_set = fake_real_df_clean.loc[test_index]

In [25]:
strat_fake_or_real_train = strat_train_set.copy()
strat_fake_or_real_test  = strat_test_set.copy()

In [26]:
X_train = strat_fake_or_real_train.copy().drop('label', 1)
y_train = strat_fake_or_real_train.copy()['label']
X_test  = strat_fake_or_real_test.copy().drop('label', 1)
y_test  = strat_fake_or_real_test.copy()['label']

In [27]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer

def preprocessing(text):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    stop = stopwords.words('english')
    tokens = [token for token in tokens if token not in stop]
    tokens = [word for word in tokens if len(word) >= 3]
    tokens = [word.lower() for word in tokens]
    lcStem = LancasterStemmer()
    tokens = [lcStem.stem(word) for word in tokens]
    preprocessed_text = ' '.join(tokens)
    return preprocessed_text

In [28]:
X_train_text_list  = list(X_train['text'])
X_train_title_list = list(X_train['title'])
X_test_text_list   = list(X_test['text'])
X_test_title_list  = list(X_test['title'])

In [29]:
X_train_text_clean = [preprocessing(text) for text in X_train_text_list]
X_train_title_clean = [preprocessing(text) for text in X_train_title_list]
X_test_text_clean = [preprocessing(text) for text in X_test_text_list]
X_test_title_clean = [preprocessing(text) for text in X_test_title_list]

In [30]:
y_train_binary = [x=='REAL' for x in list(y_train)]
y_test_binary = [x=='REAL' for x in list(y_test)]

In [31]:
'''
The code above is all cleaning.
I separated the two features Text and title. 
You can join them into a df
if you want to use multiple features analysis.
Below is the actual machine learning that I have done
so far.
'''
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=2, ngram_range=(1,2), stop_words='english', strip_accents='unicode',norm=u'l2')

X_train_tfidf = vectorizer.fit_transform(X_train_text_clean, y_train_binary)

In [32]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, y_train_binary)

In [33]:
from sklearn.model_selection import cross_val_score
cross_val_score(clf, X_train_tfidf, y_train_binary, cv=5, scoring="accuracy")

array([ 0.85024631,  0.82840237,  0.83810464,  0.84698914,  0.86080948])

In [15]:
'''
It is now up to you to use the data as you wish or change
anything.
I am still going to work on improving the current model.
'''

'\nIt is now up to you to use the data as you wish or change\nanything.\nI am still going to work on improving the current model.\n'

In [35]:
import pickle
objects = []
with (open("X_test_text_clean.pickle", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

In [36]:
objects

[["polit mak many campaign prom n't intend del but netanyahu prom monday nev agr palestin stat fit record israel prim min benjamin netanyahu talk visit construct sit har hom east jerusalem monday march 2015 day ahead legisl elect netanyahu seek four term prim min with israel fin pre-election pol point difficult road prim min benjamin netanyahu stay pow spent fin day campaign trail throwing red meat bas pro-likud robocal warn israel vot mr. netanyahu strength stand hussein obam campaign ad comp israel dock work reg hama milit cal oppon tool shadowy foreign fin strange charg giv clos tie casino billionair sheldon adelson but monday prim min del show stop vot 'll kil left peac process ston dead though prec word mean and on campaign prom vot could prob tak bank prom help tip elect bal fav would mean road palestin stat heart oslo accord sign year ago com dead end think anyon mov est palestin stat evacu territ giv territ away rad islam attack israel told nrg pro-settler new websit own mr. ad